# Milestone 1
## DSCI 525 Web and Cloud Computing
## Group 16

This notebook downloads various observed and simulated rainfall data sets from New South Wales, Australia over the period of 1889 - 2014.  The data are then combined and basic exporatory data analyses are conducted using both Python and R programming languages.

In [1]:
import re
import os
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

# Data Download
The following code chunk downloads the data used in the subsequent analyses.  The data are downloaded from 'figshare.com'.  The file 'data.zip' is saved to a local directory called 'data'.

In [3]:
%%time
%%memit
# Print out time and memory taken for downloading data

# This code is adapted from DSCI 525 lecture demonstration notebook (Gittu George, 2021,
# https://github.ubc.ca/MDS-2020-21/DSCI_525_web-cloud-comp_students/blob/master/Lectures/Lecture_1_2.ipynb)
url = f"https://api.figshare.com/v2/articles/14096681"
headers = {"Content-Type": "application/json"}
output_directory = "data/"

response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]             

for file in files:
    if file["name"] in "data.zip":
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

peak memory: 163.92 MiB, increment: 1.66 MiB
CPU times: user 4.11 s, sys: 5.67 s, total: 9.78 s
Wall time: 31.4 s


After it has been downloaded locally, 'data.zip' is extracted and stored in the 'data' directory.

In [4]:
%%time
%%memit
# Print out time and memory taken to extract data

with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

peak memory: 164.25 MiB, increment: 0.04 MiB
CPU times: user 15.1 s, sys: 2.27 s, total: 17.4 s
Wall time: 17.5 s


So annoying to load all csvs into ram, combine, then resave.  Would be much easier if we could stitch the files together directly without loading them into RAM.

# Combining Data
The following code chunk combines all of the unzipped rainfall data .csv files into a single file called 'combined_data.csv'.  This process is accomplished by creating a pandas dataframe called `full_df`, then one by one loading each .csv file and concatenating it with `full_df`.  This requires that all of the .csv files be read into a pandas dataframe variable and held in RAM at once.  In this case, this requires that almost 7 GB of data be held in RAM and manipulated.  Some computers will not be able to perform this data combining operation because they do not have sufficient RAM.  Even for systems which have sufficient RAM, performing simple operations (such as concatenation) on on a variable of this size are time consuming.  To demonstrate this, below the code chunk, we have included screen shots of the time and memory usage for the execution of this data combining operation.  To summarize, the time taken to complete this operation on each system are listed below (along with some general hardware specifications):
1.  Wall time: 7min 9s; Peak memory: 6891.53 MiB
  - Processor: i7-10510U (8 x 1.80 GHz)
  - RAM: 16 GB

In [5]:
%%time
%%memit
# Print out time and memory taken to merge and save csv files

file_names = os.listdir(output_directory)
file_names = [file for file in file_names if file[-4:]=='.csv']


cols = ['lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']
full_df = pd.DataFrame(columns = ['model']+cols)
full_df.index.rename('time', inplace=True)

for file in file_names:
    model_name = re.search('^.*(?=_daily)',file).group(0)
    full_df = pd.concat([full_df,pd.read_csv(output_directory + file, index_col=0).assign(model = model_name)])
    
full_df.to_csv(output_directory + 'combined_data.csv')

AttributeError: 'NoneType' object has no attribute 'group'

1. Processor: i7-10510U (8 x 1.80 GHz); RAM: 16 GB

![](../img/i7-10510_16GB-SP.png)